# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of this project is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports



In [1]:
 !pip install XlsxWriter

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing I need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.
 

I will use a static version of the S&P 500 constituents.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv') 
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token.


In [4]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure my API calls to IEX cloud. 

I need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 88617088,
 'calculationPrice': 'close',
 'change': -5.12,
 'changePercent': -0.03073,
 'close': 0,
 'closeSource': 'aoiifclf',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': 'u e eiidr1dnm t5aceepyl',
 'highTime': 1688754750915,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 166.58,
 'iexCloseTime': 1676010044482,
 'iexLastUpdated': 1651575142341,
 'iexMarketPercent': 0.01908941995623707,
 'iexOpen': 173.42,
 'iexOpenTime': 1667919764746,
 'iexRealtimePrice': 165.81,
 'iexRealtimeSize': 402,
 'iexVolume': 1472820,
 'lastTradeTime': 1709009613930,
 'latestPrice': 167.62,
 'latestSource': 'Close',
 'latestTime': 'April 14, 2022',
 'latestUpdate': 1702709143282,
 'latestVolume': None,
 'low': 0,
 'lowSource': '5deimrda tyi uee

## Parsing My API Call

The API call that I executed in the last code block contains all of the information required to build my equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. I need to parse it first.

In [6]:
price = data['latestPrice']
print(price)
market_cap = data['marketCap']
print(market_cap/1000000000000)

167.62
2.750813441532


## Adding My Stocks Data to a Pandas DataFrame

The next thing I need to do is add my stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe 

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [8]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
        index = my_columns
    ), ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,167.62,2750813441532,N/A


## Looping Through The Tickers in My List of Stocks

Using the same logic that I outlined above, I can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns) 
for stock in stocks['Ticker'][:1]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ], 
            index = my_columns),
        ignore_index = True
    )

In [10]:
print(final_dataframe)

  Ticker  Stock Price Market Capitalization Number of Shares to Buy
0      A       132.17           39639761528                     N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.


In [11]:
def chunks(lst,n):
    for i in range(0, len(lst), n):
        yield lst[1:1 + n]

In [12]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAL,19,12817492383,N/A
1,AAP,221.77,13516816765,N/A
2,AAPL,171.16,2769227589576,N/A
3,ABBV,167.4,295986727709,N/A
4,ABC,167.95,34007382694,N/A
...,...,...,...,...
595,CL,83.78,68955248884,N/A
596,CLX,147.55,18267658238,N/A
597,CMA,90.72,11791136460,N/A
598,CMCSA,48.21,218505395126,N/A


## Calculating the Number of Shares to Buy


In [13]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:10000000


In [16]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy,Number Of Shares to Buy
0,AAL,19,12817492383,N/A,877.0
1,AAP,221.77,13516816765,N/A,75.0
2,AAPL,171.16,2769227589576,N/A,97.0
3,ABBV,167.4,295986727709,N/A,99.0
4,ABC,167.95,34007382694,N/A,99.0
...,...,...,...,...,...
595,CL,83.78,68955248884,N/A,198.0
596,CLX,147.55,18267658238,N/A,112.0
597,CMA,90.72,11791136460,N/A,183.0
598,CMCSA,48.21,218505395126,N/A,345.0


## Formatting My Excel Output

I will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. 

### Initializing our XlsxWriter Object

In [17]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. I need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [18]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

I can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [19]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [20]:
writer.save()